# Read HDF files from results database
This notebook shows how to extract results from hdf (`.h5`) files that are already present in the `\results_database\` folder that is present in each benchmark folder (in `\models\`). These files can store either experimental or computational results of the same experiment/simulation. The `\results_database\` folder of the should have all the same structure and same ways to extract the data (e.g. the name of the tally).

For this tutorial notebook the `\results_database\` folder is replaced with the `\example_database\` folder. The `\example_database\` folder contains some of the actual results from the **fng-streaming** benchmark as example.

In [1]:
import openmc_fusion_benchmarks as ofb

First we need to instantiate a `ResultsFromDatabase` object by providing the name of the file to read (`filename`) and its path (`path`).

In [2]:
experiment_file = ofb.ResultsFromDatabase(filename='experiment.h5', path='example_database')
mcnp_fendl1_file = ofb.ResultsFromDatabase(filename='mcnp_fendl1.h5', path='example_database')

Usually the file is read from a **postprocessing** notebook that is in the benchmark/model folder with contains the `\results_database\` folder. Therefore, the `path` argument defaults to a `\results_database\` folder.

### Extract info
If present, it is possible to extract and print some information regarding the experiment/simulation that produced the results present in the hdf file. Typical information can be:
- `when` the experiment/simulation have been performed
- `Where` the experiment/simulation have been performed (either place or institution)
- `literature_info` regarding the experiment/model/simulation (most likely the link to a paper or report)
- `code_version` if the hdf file refers to simulation results, useful info for V&V purposes
- `xs_library` (nuclear data library name and version) if the hdf file refers to simulation results, useful info for V&V purposes

Example of experimental information extraction:

In [3]:
when = experiment_file.when
where = experiment_file.where
literature = experiment_file.literature_info

print(f'When: {when}')
print(f'Where: {where}')
print(f'Literature: {literature}')

When: ['1997-1998']
Where: ['ENEA, Frascati (Italy)']
Literature: ['https://doi.org/10.1016/S0920-3796(00)00205-2']


It is also possible to print all the information at once with the `print_all_info` method:

In [4]:
experiment_file.print_all_info()

Info:
 When: ['1997-1998']
 Where: ['ENEA, Frascati (Italy)']
 Code version: n/a
 XS library: n/a
 Literature: ['https://doi.org/10.1016/S0920-3796(00)00205-2']



Similarly, for hdf files containing numerical simulation results:

In [5]:
cv = mcnp_fendl1_file.code_version
xslib = mcnp_fendl1_file.xs_library
literature = mcnp_fendl1_file.literature_info

mcnp_fendl1_file.print_all_info()

Info:
 When: n/a
 Where: n/a
 Code version: MCNP-4A/B
 XS library: FENDL-1.0
 Literature: ['https://doi.org/10.1016/S0920-3796(00)00205-2']



In the case of hdf file containing simulation results it is possible to print just the `code_info`. Namely the `code_version` and `xs_library`:

In [6]:
mcnp_fendl1_file.print_code_info()

Code version:MCNP-4A/B
 XS library: FENDL-1.0



### Extract a tally

In order to extract a tally from a hdf file it is necessary to know its exact name. The `ResultsFromDatabase` object has a `list_tallies()` method that helps identifying the names of all the tallies available:

In [7]:
experiment_file.list_tallies()

<KeysViewHDF5 ['nuclear_heating', 'rr_offaxis_al27', 'rr_offaxis_au197', 'rr_offaxis_nb93', 'rr_offaxis_ni58', 'rr_onaxis1_al27', 'rr_onaxis1_au197', 'rr_onaxis1_nb93', 'rr_onaxis1_ni58', 'rr_onaxis2_al27', 'rr_onaxis2_au197', 'rr_onaxis2_nb93', 'rr_onaxis2_ni58']>


We chose, as example, the tally named `rr_offaxis_al27` and extract its results in a Pandas DataFrame format with the `get_tally_dataframe` method:

In [8]:
results = experiment_file.get_tally_dataframe(tally_name='rr_onaxis1_al27')

results

,Shield depth (cm),mean,std. dev.
0,0.25,2.870000e-04,1.004500e-05
1,12.95,3.540000e-05,1.239000e-06
2,25.95,1.230000e-05,4.920000e-07
3,38.65,6.060000e-06,2.424000e-07
4,43.82,4.650000e-06,1.860000e-07
5,46.35,3.280000e-06,1.902400e-07
6,53.3,1.020000e-06,6.528000e-08
7,60.05,3.020000e-07,1.963000e-08
8,66.9,7.980000e-08,5.665800e-09
9,73.9,2.460000e-08,1.918800e-09


It is possible to view single columns of the dataframe by calling the column name (names listed in the bottom line of the previous):

In [9]:
print(results['mean'])
print(results['std. dev.'])

0     2.870000e-04
1     3.540000e-05
2     1.230000e-05
3     6.060000e-06
4     4.650000e-06
5     3.280000e-06
6     1.020000e-06
7     3.020000e-07
8     7.980000e-08
9     2.460000e-08
10    7.310000e-09
11    2.140000e-09
12    1.070000e-09
Name: mean, dtype: float64
0     1.004500e-05
1     1.239000e-06
2     4.920000e-07
3     2.424000e-07
4     1.860000e-07
5     1.902400e-07
6     6.528000e-08
7     1.963000e-08
8     5.665800e-09
9     1.918800e-09
10    6.871400e-10
11    2.396800e-10
12    1.337500e-10
Name: std. dev., dtype: float64


Typical experimental or statistical results (e.g. Monte Carlo simulation) are provided with the `mean` and `std. dev.` columns.
Where the `std. dev.` column provides the **standard deviation absolute values (not relative)**.

Strings need to be decoded:

In [10]:
print([el.decode() for el in results['Shield depth (cm)']])

AttributeError: 'str' object has no attribute 'decode'